In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import sklearn

df=pd.read_csv('./dataset/train.csv',encoding='utf-8') 
test=pd.read_csv('./dataset/test.csv',encoding='utf-8') 
df.head()

,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales
0,1,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,8.106,False,1643690.90
1,2,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,8.106,True,1641957.44
2,3,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,8.106,False,1611968.17
3,4,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,8.106,False,1409727.59
4,5,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,8.106,False,1554806.68


In [2]:
# Train 데이터 Date, IsHoliday 컬럼 변환 
df['Date']=pd.to_datetime(df['Date']) 
df['Day']=df['Date'].dt.day 
df['Month']=df['Date'].dt.month 
df['Year']=df['Date'].dt.year 
df['IsHoliday']=df['IsHoliday'].astype('int64') 

df.head()

,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales,Day,Month,Year
0,1,1,2010-05-02,42.31,2.572,NaN,NaN,NaN,NaN,NaN,8.106,0,1643690.90,2,5,2010
1,2,1,2010-12-02,38.51,2.548,NaN,NaN,NaN,NaN,NaN,8.106,1,1641957.44,2,12,2010
2,3,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,8.106,0,1611968.17,19,2,2010
3,4,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,8.106,0,1409727.59,26,2,2010
4,5,1,2010-05-03,46.50,2.625,NaN,NaN,NaN,NaN,NaN,8.106,0,1554806.68,3,5,2010


In [3]:
# test 데이터 Date, IsHoliday 컬럼 변환 
test['Date']=pd.to_datetime(test['Date']) 
test['Day']=test['Date'].dt.day 
test['Month']=test['Date'].dt.month 
test['Year']=test['Date'].dt.year 
test['IsHoliday']=test['IsHoliday'].astype('int64') 

test.head()

,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Day,Month,Year
0,1,1,2012-05-10,68.55,3.617,8077.89,NaN,18.22,3617.43,3626.14,6.573,0,10,5,2012
1,2,1,2012-12-10,62.99,3.601,2086.18,NaN,8.11,602.36,5926.45,6.573,0,10,12,2012
2,3,1,2012-10-19,67.97,3.594,950.33,NaN,4.93,80.25,2312.85,6.573,0,19,10,2012
3,4,1,2012-10-26,69.16,3.506,2585.85,31.75,6.00,1057.16,1305.01,6.573,0,26,10,2012
4,5,2,2012-05-10,70.27,3.617,6037.76,NaN,10.04,3027.37,3853.40,6.170,0,10,5,2012


In [4]:
df['Weekly_Sales'] = np.log1p(df['Weekly_Sales']) # Log 변환

from pycaret.regression import *

In [5]:
automl=setup(data=df, target='Weekly_Sales', ignore_features=['id','Date'], n_jobs=-1,
             normalize=True, numeric_imputation='zero', silent=True, 
             remove_multicollinearity=True, fold_shuffle=True, fold=20, use_gpu=True,
             combine_rare_levels = True, ignore_low_variance = True)

,Description,Value
0,session_id,7929
1,Target,Weekly_Sales
2,Original Data,"(6255, 16)"
3,Missing Values,True
4,Numeric Features,10
5,Categorical Features,3
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(4378, 25)"


AttributeError: 'Simple_Imputer' object has no attribute 'fill_value_categorical'

In [6]:
# 7개 모델 선택
best_7 = compare_models(n_select=7, round=4, sort='RMSE', fold=20)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.0564,0.0058,0.0759,0.9833,0.0052,0.0041,2.0885
lightgbm,Light Gradient Boosting Machine,0.0615,0.0079,0.0880,0.9774,0.0060,0.0045,1.4650
rf,Random Forest Regressor,0.0674,0.0125,0.1104,0.9643,0.0074,0.0049,1.5005
et,Extra Trees Regressor,0.0731,0.0125,0.1111,0.9643,0.0075,0.0053,1.4555
dt,Decision Tree Regressor,0.0844,0.0230,0.1477,0.9345,0.0100,0.0061,0.0355
gbr,Gradient Boosting Regressor,0.1623,0.0407,0.2014,0.8836,0.0138,0.0119,0.6145
ada,AdaBoost Regressor,0.3953,0.2071,0.4547,0.4083,0.0311,0.0290,0.3215
knn,K Neighbors Regressor,0.4243,0.2740,0.5229,0.2167,0.0358,0.0312,0.0440
lr,Linear Regression,0.4491,0.2898,0.5378,0.1721,0.0369,0.0331,0.0140
br,Bayesian Ridge,0.4496,0.2898,0.5378,0.1721,0.0369,0.0331,0.0090


In [ ]:
tuned_7=[tune_model(i) for i in best_7]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.1059,0.0250,0.1581,0.9322,0.0106,0.0077
1,0.0917,0.0180,0.1341,0.9528,0.0091,0.0067
2,0.0880,0.0166,0.1288,0.9506,0.0088,0.0065
3,0.1039,0.0232,0.1522,0.9335,0.0103,0.0075
4,0.1029,0.0230,0.1516,0.9345,0.0102,0.0075
5,0.0792,0.0122,0.1105,0.9634,0.0075,0.0058
6,0.1097,0.0311,0.1764,0.9105,0.0119,0.0080
7,0.0881,0.0177,0.1329,0.9444,0.0090,0.0064
8,0.1001,0.0239,0.1546,0.9386,0.0105,0.0073
9,0.0941,0.0173,0.1315,0.9523,0.0089,0.0069


IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,07:18:20
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Extra Trees Regressor


,MAE,MSE,RMSE,R2,RMSLE,MAPE


Fitting 20 folds for each of 10 candidates, totalling 200 fits


In [ ]:
blender=blend_models(estimator_list=tuned_7, optimize='RMSE')

In [ ]:
final=finalize_model(blender)
pred=predict_model(final, data=test)

In [ ]:
output=pd.DataFrame({'id':test['id'],'Weekly_Sales':pred['Label']})
output.head()

In [ ]:
# output.to_csv('submission_CZ_6.csv',index=False)

In [ ]:
# 7개 모델 스택
stacker=stack_models(tuned_7, optimize='RMSE',fold=20)

In [ ]:
s_pred=predict_model(stacker, data=test)

In [ ]:
s_output=pd.DataFrame({'id':test['id'],'Weekly_Sales':s_pred['Label']})
s_output.head()

In [ ]:
# s_output.to_csv('submission_CZ_8.csv',index=False)